In [142]:
#for aoi in list_aois[:1]:
    #shape = shapely.to_geojson(tiles.loc[aoi,"geometry"])
    #f = write_json(shape)
#    tile = tiles[tiles['random']==aoi]
    #print(type(tile))
#    tile.to_file('temp.geojson')
    

<class 'geopandas.geodataframe.GeoDataFrame'>


This code works through the GEDI subsetter version 0.6.0. This converts a geojson of tiles over a study area to temp geojson files to feed into the subsetter

In [2]:
#Open the MAAP host
from maap.maap import MAAP
maap = MAAP(maap_host='api.ops.maap-project.org')


In [3]:
#pip install geopandas
#pip install shapely
#Import numpy for making column inputs easier
import numpy as np
import geopandas as gpd
import shapely

/opt/conda/lib/python3.7/site-packages/geopandas/_compat.py:115: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.8.1-CAPI-1.13.3). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


In [4]:
#Load in Tiled AOI
tiles = gpd.read_file("https://maap-ops-workspace.s3.amazonaws.com/shared/abarenblitt/EEZBounds/WestAfricaGrid.geojson")

In [5]:
import os

result_folder = "../aoi/WestAfricaGrid"

if not os.path.exists(result_folder):
    os.makedirs(result_folder)

for rownum, subframe in tiles.groupby("random"):
    filename = os.path.join(result_folder, f"{rownum}.geojson")
    subframe.to_file(filename, driver='GeoJSON')

len(os.listdir(result_folder))

151

In [156]:
#os.listdir(result_folder)

In [61]:
#Set index to the correct column name, in this case "random"
#tiles.set_index('random',inplace=True)
#print(tiles.head())

In [62]:
#for col in tiles.columns:
#    print(col)

In [63]:
#tiles['random_value'] = tiles.index

In [64]:
#Test a single tile
#tiles.loc[0.6855097362903453]

Use "random" column for ID to generate features to tile over

In [65]:
#List out the individual aois
#list_aois = tiles.index.unique().tolist() #tiles[tiles.columns[1]].values.tolist()
#print(list_aois)

In [66]:
#aoiTest=shapely.to_geojson(tiles.loc[0.6855097362903453,"geometry"])
#print(aoiTest)

In [6]:
#Create empty variables to fill with all 29 appearances of cover_z* and pai_z* 
variables = []
for n in np.arange(0, 30,1):
    variables.append('cover_z' + str(n))
    variables.append('pai_z' + str(n))


For several AOI's run this code:

In [7]:
#Set up job results to print output during run

import xml.etree.ElementTree as ET
from urllib.parse import urlparse

def job_status_for(job_id: str) -> str:
    response = maap.getJobStatus(job_id)
    response.raise_for_status()
    
    root = ET.fromstring(response.text)
    status_element = root.find('.//{http://www.opengis.net/wps/2.0}Status')
    
    return status_element.text

def job_result_for(job_id: str) -> str:
    response = maap.getJobResult(job_id)
    response.raise_for_status()
    
    root = ET.fromstring(response.text)

    return root.find('.//{http://www.opengis.net/wps/2.0}Data').text

def to_job_output_dir(job_result_url: str) -> str:
    url_path = urlparse(job_result_url).path
    # The S3 Key is the URL path excluding the `/{username}` prefix
    s3_key = "/".join(url_path.split("/")[2:])

    return f"/projects/my-private-bucket/{s3_key}"

In [41]:
#Import geojson and tempfile for creating temp files for the subsetter

#from geojson import Point, Feature, FeatureCollection, dump

#import tempfile

In [8]:
#Set up input dictionary and import backoff to display run results
#First part of the for loop uses the "random" column to search through the tiles geojson to 
#create temp files one at a time to set the AOI
#################################

for aoi in os.listdir(result_folder):
    if aoi.endswith("geojson"):
        inputs = dict(
           aoi=f"https://maap-ops-workspace.s3.amazonaws.com/shared/abarenblitt/aoi/WestAfricaGrid/{aoi}",
           doi="L2B",
           lat="geolocation/lat_lowestmode",
           lon="geolocation/lon_lowestmode",
           beams="all",
           columns="rh100, cover,land_cover_data/landsat_treecover, pai,fhd_normal,"+",".join(variables),
           query="l2a_quality_flag == 1 and l2b_quality_flag == 1 and sensitivity > 0.95",
           limit = 10,
           temporal="-",
           output="-"
        )
        result = maap.submitJob(
            identifier="gedi-subset",
            algo_id="gedi-subset_ubuntu",
            version="0.6.0",
            queue="maap-dps-worker-32gb",
            username="abarenblitt",
            **inputs
        )

        job_id = result["job_id"]
        print(job_id)
        
try:
    import backoff
except:
    job_status = job_status_for(job_id)
else:
    # Check job status every 2 minutes
    @backoff.on_predicate(
        backoff.constant,
        lambda status: status not in ["Deleted", "Succeeded", "Failed"],
        interval=120,
    )
    def wait_for_job(job_id: str) -> str:
        return job_status_for(job_id)

    job_status = wait_for_job(job_id)
    job_status

a19e7b6b-440b-4688-9e6e-8ba0f6bc237c
b987653b-a57b-467f-8967-37d0400ab0e3
326fe9d7-06e2-4f12-9e80-b47805d10c6f
2e1d8f2f-235c-491c-8a2f-640a60045067
aea0293d-bcd3-4c5c-b06b-862d336b1589
e56491bb-2a3f-442d-9f6e-c920463cb38b
d760f61c-725a-405e-bcae-21c4eb665efd
ebcc6282-c8b0-4523-a6ef-376b3459d1fb
b0245f46-6d45-41a9-9787-a3ecd16201fe
94308961-ac8b-47b1-a6f8-8144a6421432
fedf7254-7743-4600-bc39-f5ed48316aef
97dc1b94-b95c-44fa-938a-3d806e2cb6ed
694a4486-98e1-41c8-add6-ae2b81802470
c259b1a7-077c-4a1e-8a81-2c285a4a9f68
245680c6-df15-425d-8645-2650d56f8b09
9e64508c-137f-4433-9d4a-bb2830929b6b
98dc8e9d-1c36-4e34-8432-eada2b7aaf90
424112fb-1edf-4847-bf82-9280f33f9e90
5786b20f-963d-4efe-a08d-8e421b5aab57
f29a486c-23dd-4f73-bb5e-ebabf65e83ea
cf2947ad-3b5f-43da-9f5c-9b0bbca717c1
a8d2c948-4951-42c1-bc70-befcebcdaffd
2fc7d95f-36da-4520-a0f0-e878009880e2
060aa6fc-d745-4e3b-896e-8254506c5bd6
9150e89b-ffcf-4026-8738-ab1db34c569a
e3c06766-5fdb-4f0d-a280-c1b8d12ee46c
9511c54e-8f1a-4461-80d7-868c2e9dc908
3

In [4]:
job_id = "40accbd5-8919-4129-bbf0-2c125345a463"

job_status

NameError: name 'job_status' is not defined

In [165]:
#Get job result full text to bug Alex with
 
maap.getJobResult(job_id).text

'<wps:Result xmlns:ows="http://www.opengis.net/ows/2.0" xmlns:schemaLocation="http://schemas.opengis.net/wps/2.0/wps.xsd" xmlns:wps="http://www.opengis.net/wps/2.0" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"><wps:JobID>fcf5e7d3-3343-43db-9fc5-71406065969f</wps:JobID><wps:Output id="traceback"><wps:Data>Traceback (most recent call last):\n  File "/home/ops/verdi/ops/hysds-0.3.11/hysds/utils.py", line 139, in download_file\n    osaka.main.get(url, cache_dir, params=params)\n  File "/home/ops/verdi/ops/osaka-1.0.1/osaka/main.py", line 86, in get\n    noclobber=noclobber,\n  File "/home/ops/verdi/ops/osaka-1.0.1/osaka/main.py", line 128, in transfer\n    noclobber=noclobber,\n  File "/home/ops/verdi/ops/osaka-1.0.1/osaka/transfer.py", line 128, in transfer\n    raise err\n  File "/home/ops/verdi/ops/osaka-1.0.1/osaka/transfer.py", line 106, in transfer\n    metrics = self.transfer_uri(source, shandle, dest, dhandle)\n  File "/home/ops/verdi/ops/osaka-1.0.1/osaka/transfer.py", li